In [11]:
import pandas as pd
import datetime as dt
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
import chart_studio.plotly as py
import plotly.figure_factory as ff
df = pd.read_csv('data/opportunities.csv',low_memory=False)

In [12]:
df.loc[df['Opportunity_ID'] == 11112,['Opportunity_Name','Total_Amount','Total_Taxable_Amount','Stage']]

,Opportunity_Name,Total_Amount,Total_Taxable_Amount,Stage
16063,Opportunity_Name_5000,1.320000e+09,11867301.99,Proposal


In [13]:
df.loc[df['Opportunity_Name'] == "Opportunity_Name_5000",['Opportunity_ID']]

,Opportunity_ID
16063,11112


In [14]:
#funcion para calcular la conversion a dolar
def conversion(x):
    if (x["Total_Taxable_Amount_Currency"] == "USD"):
        return 1
    elif (x["Total_Taxable_Amount_Currency"] == x["ASP_Currency"] and float(x["ASP"]) != 0):  
        return x["ASP_(converted)"] / float(x["ASP"])
    else:
        return None

In [15]:
#creo condicion donde para filtrar registros donde no tenga manera de convertir a moneda el total taxeable
condition = (df['Total_Taxable_Amount_Currency'] != df['Total_Amount_Currency'])\
& (df['Total_Taxable_Amount_Currency'] != df['ASP_Currency'])\
& (df['Total_Taxable_Amount_Currency'] != df['ASP_(converted)_Currency'])

## Top 10 de oportunidades exitosas ordenada por el monto total 

In [16]:
#filtro por oportunidades exitosas
df_close_won = df.loc[df['Stage'] == 'Closed Won']
#quedan 5 registros afuera de los mas de 9000
df_close_won_filtered = df_close_won.loc[condition!= True,['Opportunity_Name','Opportunity_ID','Price','Currency','Total_Amount','Total_Amount_Currency','ASP','ASP_Currency','ASP_(converted)','ASP_(converted)_Currency','Total_Taxable_Amount','Total_Taxable_Amount_Currency','Planned_Delivery_Start_Date']]
df_close_won_filtered

,Opportunity_Name,Opportunity_ID,Price,Currency,Total_Amount,Total_Amount_Currency,ASP,ASP_Currency,ASP_(converted),ASP_(converted)_Currency,Total_Taxable_Amount,Total_Taxable_Amount_Currency,Planned_Delivery_Start_Date
1,Opportunity_Name_12600,1,None,None,48230.0,EUR,0.530,EUR,0.59948,USD,48230.00,EUR,1/18/2016
2,Opportunity_Name_469,2,None,None,83865.6,USD,0.480,USD,0.48000,USD,83865.60,USD,1/25/2016
7,Opportunity_Name_8227,7,None,None,50688.0,USD,0.480,USD,0.48000,USD,50688.00,USD,1/25/2016
23,Opportunity_Name_7036,11,None,None,1621851.0,USD,0.539,USD,0.53900,USD,1621851.00,USD,1/9/2016
63,Opportunity_Name_4062,20,None,None,103350.0,EUR,0.530,EUR,0.59948,USD,103350.00,EUR,1/18/2016
...,...,...,...,...,...,...,...,...,...,...,...,...,...
16937,Opportunity_Name_4060,12797,None,None,413400.0,EUR,0.530,EUR,0.59948,USD,528918.75,EUR,2/1/2016
16939,Opportunity_Name_7202,12799,None,None,401700.0,EUR,0.515,EUR,0.58251,USD,401700.00,EUR,4/1/2016
16942,Opportunity_Name_9245,12801,None,None,103350.0,EUR,0.520,EUR,0.58817,USD,299715.00,EUR,3/21/2016
16943,Opportunity_Name_9245,12801,None,None,93015.0,EUR,0.520,EUR,0.58817,USD,299715.00,EUR,4/4/2016


In [17]:
#agrego el valor de conversion a moneda dolar
df_close_won_filtered["convertion"] = df_close_won_filtered.apply(conversion, axis=1)
df_close_won_filtered["convertion"].value_counts()

1.000000    3593
1.131095     169
0.008990     141
1.131091     139
0.008990     136
            ... 
1.131108       1
0.707623       1
0.707621       1
0.009049       1
0.008935       1
Name: convertion, Length: 186, dtype: int64

In [18]:
#Convierto el total tax amount a moneda dolar para poder comparar
df_close_won_filtered["Total Taxable Amount (USD)"] = df_close_won_filtered["Total_Taxable_Amount"] * df_close_won_filtered["convertion"]

#agrupo por oportunidad
df_opportunity_Name = df_close_won_filtered.groupby(["Opportunity_Name","Opportunity_ID"]).agg({"Total Taxable Amount (USD)":"mean"})\
.reset_index().nlargest(10,"Total Taxable Amount (USD)")

#Preparo la tabla a mostrar
df_opportunity_table = df_opportunity_Name.loc[:,["Opportunity_Name","Total Taxable Amount (USD)"]]\
.rename(columns={"Opportunity_Name": "Opportunity"})

In [19]:
table = ff.create_table(df_opportunity_table)
py.iplot(table)

In [29]:
#agrupo por oportunidad y vendedor y me quedo con su estado
df_aux = df.loc[:,["Opportunity_Owner","Stage","Opportunity_Name"]].groupby(["Opportunity_Name","Opportunity_Owner"])\
.agg({'Stage':'last'}).reset_index()
#agrupo solo por vendedor y cuento la cantidad de apariciones de los estados para ese vendedor
df_owner = df_aux.groupby("Opportunity_Owner")["Stage"].value_counts().to_frame()
#si no tiene cierto estado para alguna oportunidad lleno con 0
df_owner2 = df_owner.unstack().fillna(0)
#renombro las columnas
df_owner2.columns = ["Closed Lost","Closed Won","Negotiation","Proposal","Qualification"]
#calculo el total de oportunidades para cada vendedor
df_owner2["Total Oportunities"] = df_owner2.agg('sum',axis="columns")
#calculo la relacion entre oportunidades exitosas y totales para cada vendedor
df_owner2["Win/Total"] = df_owner2["Closed Won"] / df_owner2["Total Oportunities"]
#calculo la relacion entre oportunidades perdidas y totales para cada vendedor
df_owner2["Lost/Total"] = df_owner2["Closed Lost"] / df_owner2["Total Oportunities"]
#calculo la relacion entre oportunidades exitosas y pedidas para cada vendedor
df_owner2["Win/Lost"] = df_owner2["Closed Won"] / df_owner2["Closed Lost"]
#ordeno por la relacion de exitosas sobre perdidas de manera descendente
df_owner2 = df_owner2.sort_values(by = "Win/Total",ascending = False).reset_index()
#filtro los vendedores que tengan menos oportunidades que el promedio de oportunidades
df_final_owner = df_owner2.loc[df_owner2["Total Oportunities"] >= df_owner2["Total Oportunities"].mean()]
df_final_owner

,Opportunity_Owner,Closed Lost,Closed Won,Negotiation,Proposal,Qualification,Total Oportunities,Win/Total,Lost/Total,Win/Lost
1,Person_Name_19,12.0,399.0,0.0,0.0,0.0,411.0,0.970803,0.029197,33.250000
4,Person_Name_32,141.0,469.0,0.0,0.0,1.0,611.0,0.767594,0.230769,3.326241
7,Person_Name_64,108.0,231.0,4.0,0.0,0.0,343.0,0.673469,0.314869,2.138889
8,Person_Name_66,63.0,124.0,0.0,4.0,0.0,191.0,0.649215,0.329843,1.968254
9,Person_Name_50,356.0,595.0,0.0,1.0,6.0,958.0,0.621086,0.371608,1.671348
13,Person_Name_13,322.0,434.0,1.0,0.0,0.0,757.0,0.573316,0.425363,1.347826
14,Person_Name_8,334.0,429.0,0.0,0.0,0.0,763.0,0.562254,0.437746,1.284431
15,Person_Name_23,117.0,149.0,0.0,0.0,0.0,266.0,0.560150,0.439850,1.273504
16,Person_Name_49,106.0,123.0,0.0,1.0,0.0,230.0,0.534783,0.460870,1.160377
20,Person_Name_18,353.0,329.0,0.0,0.0,0.0,682.0,0.482405,0.517595,0.932011


In [22]:
data = df_owner.rename(columns={"Stage": "Count"}).reset_index()
data = data.loc[data["Stage"].isin(['Closed Won','Closed Lost']),:]
data["Total"] = pd.to_numeric(data.groupby("Opportunity_Owner").transform("sum")["Count"],downcast="float")
data["Win/Total"] = data["Count"] / data["Total"]
data = data.nlargest(20,"Total")
data

,Opportunity_Owner,Stage,Count,Total,Win/Total


In [ ]:
fig,ax=plt.subplots(figsize=(30,12))

plt.grid(True)
g = sns.barplot(ax=ax,data=data,x='Opportunity_Owner',y='Count',palette='YlGnBu',alpha=1,hue='Stage')
g.set_title('Cantidad de oportunidades segun su estado para los vendedores con mas oportunidades\n',size=35)
g.set_xlabel('Vendedor',size=25)
g.set_ylabel('Cantidad de oportunidades',size=25)
g.legend(title='Stage')

In [ ]:
show = df_final_owner.head(10)
fig,ax=plt.subplots(figsize=(10,5))
sns.barplot(data=show,x='Win/Total',y='Opportunity_Owner',palette="spring")
plt.grid(True)
plt.xlabel('Relacion oportunidad exitosas sobre totales',size=15)
plt.ylabel('')
plt.title('Top 10 vendedores con mejor promedio de oportunidades exitosas\n',size=22)


In [ ]:
df_close_won_owner = df_close_won.loc[condition!= True,['Opportunity_Name','Opportunity_Owner','Total_Amount','Total_Amount_Currency','ASP','ASP_Currency','ASP_(converted)','ASP_(converted)_Currency','Total_Taxable_Amount','Total_Taxable_Amount_Currency']]
#agrego el valor de conversion a moneda dolar
df_close_won_owner["convertion"] = df_close_won_owner.apply(conversion, axis=1)
#Convierto el total tax amount a moneda dolar para poder comparar
df_close_won_owner["Total Taxable Amount (USD)"] = df_close_won_owner["Total_Taxable_Amount"] * df_close_won_owner["convertion"]
#agrupo por oportunidad y vendedor para obtener el promedio del monto total 
df_close_won_owner = df_close_won_owner.groupby(["Opportunity_Name","Opportunity_Owner"]).agg({"Total Taxable Amount (USD)":"mean"}).reset_index()
#agrupo por vendedor para obtener la suma de los promedios de los montos totales de sus oportunidades exitosas 
df_close_won_owner = df_close_won_owner.groupby(["Opportunity_Owner"]).agg({"Total Taxable Amount (USD)":"sum"}).reset_index()
#me quedo con los 10 vendedores con mayor dinero obtenido
df_close_won_owner = df_close_won_owner.nlargest(10,"Total Taxable Amount (USD)")
df_close_won_owner

In [ ]:
fig,ax=plt.subplots(figsize=(10,5))
sns.barplot(data=df_close_won_owner,x='Total Taxable Amount (USD)',y='Opportunity_Owner',palette="magma")
plt.grid(True)
plt.xlabel('Total Taxable Amount (USD)',size=15)
plt.ylabel(' ')
plt.title('Vendedores con mayor dinero generado con oportunidades exitosas\n',size=22)

In [ ]:
df_acc_opp = df.loc[:,["Opportunity_Name","Stage","Opportunity_Owner","Account_Owner"]]
df_acc_opp["Opportunity_Owner is Account_Owner"] = df_acc_opp["Opportunity_Owner"] == df_acc_opp["Account_Owner"]
df_acc_opp = df_acc_opp.groupby(["Opportunity_Name","Stage"]).agg({"Opportunity_Owner is Account_Owner":"first"}).reset_index()
df_acc_opp = df_acc_opp.groupby(["Stage","Opportunity_Owner is Account_Owner"]).count().reset_index()
df_acc_opp.columns = ["Stage","Opportunity_Owner equals Account_Owner","Count"]
df_acc_opp

In [ ]:
df_acc_opp_data = df_acc_opp.loc[df_acc_opp["Stage"].isin(['Closed Won','Closed Lost']),:]

fig,ax=plt.subplots(figsize=(20,12))
plt.grid(True)
g = sns.barplot(ax=ax,data=df_acc_opp_data,x='Stage',y='Count',palette='Dark2',hue='Opportunity_Owner equals Account_Owner')
g.set_title('Oportunidades cuando el vendedor responsable de la cuenta comercial \n es el mismo que el responsable de la cuenta del cliente\n',size=35)
g.set_xlabel('Stage',size=30)
g.set_ylabel('Cantidad de oportunidades',size=30)
g.legend(title='Account owner = Oportunity owner',fontsize = '10',title_fontsize='20',loc='upper left')

In [ ]:
df.loc[df["Opportunity_Name"] == "Opportunity_Name_9990",["Opportunity_Type"]]
df_opp_type = df.loc[:,["Opportunity_Name","Opportunity_Type","Stage"]].groupby(["Opportunity_Name","Opportunity_Type"])["Stage"].value_counts().to_frame()
df_opp_type["Stage"] = 1
df_opp_type["total"] = df_opp_type["Stage"].sum()
df_opp_type = df_opp_type.rename(columns={"Stage": "Count"}).reset_index()

df_opp_type_show = df_opp_type.groupby("Opportunity_Type").agg({"Count":"sum","total":"mean"}).reset_index()
df_opp_type_show.columns = ["Opportunity_Type","Count","Total opportunities"]
df_opp_type_show["Count/Total"] = df_opp_type_show["Count"] / df_opp_type_show["Total opportunities"]
df_opp_type_show["Percentage"] = df_opp_type_show["Count/Total"] * 100
df_opp_type_show

In [ ]:
data_show_pie = df_opp_type_show.round(2).nlargest(8,"Percentage")
fig1,ax1 = plt.subplots(figsize=(10,10))

labels=data_show_pie['Opportunity_Type']+"  (" + (data_show_pie['Percentage']).astype(str) + "%)"
explode = (0,0,0,0.1,0.25,0.25,0.25,0.4)
colors = [] #podemos cambiar los colores

ax1.axis('equal')
ax1.pie(data_show_pie['Percentage'],labels=labels,explode=explode)
plt.title('Porcentaje de oportunidades segun su tipo',size=18)
plt.show()

In [ ]:
data_opp_type_show_bar = df_opp_type_show.round(2).nlargest(10,"Percentage")

fig,ax=plt.subplots(figsize=(30,12))
plt.yscale('log')
plt.grid(True)
g = sns.barplot(ax=ax,data=data_opp_type_show_bar,x='Opportunity_Type',y='Count',palette='RdPu_r',alpha=1)
g.set_title('Cantidad de oportunidades segun su tipo (log)\n',size=35)
g.set_xlabel('Tipo de oportunidad',size=25)
g.set_ylabel('Cantidad de oportunidades (log)',size=25)

In [ ]:
data_opp_type_show_bar = df_opp_type_show.round(2).nlargest(10,"Percentage")

fig,ax=plt.subplots(figsize=(30,12))
plt.grid(True)
g = sns.barplot(ax=ax,data=data_opp_type_show_bar,x='Opportunity_Type',y='Count',palette='PuRd_r',alpha=1)
g.set_title('Cantidad de oportunidades segun su tipo\n',size=35)
g.set_xlabel('Tipo de oportunidad',size=25)
g.set_ylabel('Cantidad de oportunidades',size=25)

In [ ]:
df_opportunity_table.head()